<a href="https://colab.research.google.com/github/sheldor07/Essay-Evaluator-Research-Assistantship/blob/main/topic_classification_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
from transformers import pipeline

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/essayData.csv')

# Initialize the zero-shot classification pipeline
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")


In [ ]:
df.head()

,Essay,Section,ABSTRACT,terms
0,1,1,Four in five Singaporeans say that they can co...,humanities
1,1,1,While the terms ÃfalsehoodsÃ and Ãfake news...,humanities
2,1,2,The digital era is characterised by easy creat...,humanities
3,1,2,It is important to be critical of fake news du...,"sciences,humanities"
4,1,3,Fake news has played an enormous role during C...,humanities


In [ ]:

# Create an empty list to store the predictions
predictions = []
df_random = df.sample(n=10)
# Iterate through each row of the DataFrame
for index, row in df_random.iterrows():
    # Get the value from the "Section" column
    sequence_to_classify = row['ABSTRACT']
    # Define the candidate labels
    candidate_labels = ['technology', 'business', 'humanities', 'sciences']

    # Perform zero-shot classification and get the predicted label
    prediction = classifier(sequence_to_classify, candidate_labels)

    # Add the predicted label to the list
    predictions.append(prediction['scores'])
    print(f'row :{row}/10')

# Add the predictions as a new column in the DataFrame
df_random['Predicted_Label'] = predictions

# Save the updated DataFrame to a new CSV file
df_random.to_csv('predicted_file_random.csv', index=False)


row :Essay                                                             10
Section                                                            6
ABSTRACT           A nationwide smart water grid which monitors w...
terms                                            sciences,technology
Predicted_Label    [0.8601968288421631, 0.05572433024644852, 0.04...
Word_Labels                                             [technology]
Name: 6, dtype: object/10
row :Essay                                                              2
Section                                                            8
ABSTRACT           Inferable from Plastic BankÃs immense virtuou...
terms                                                       business
Predicted_Label    [0.9386627674102783, 0.024442708119750023, 0.0...
Word_Labels                                             [technology]
Name: 4, dtype: object/10
row :Essay                                                              3
Section                             

In [ ]:

# Dictionary mapping indices to labels
labels_dict = {
    0: 'technology',
    1: 'business',
    2: 'humanities',
    3: 'sciences'
}


In [ ]:
import pandas as pd
import ast

# Read the CSV file into a DataFrame
df = pd.read_csv('/content/predicted_file.csv')


# Create an empty list to store the word labels
word_labels = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    labels = ast.literal_eval(row['Predicted_Label'])
    word_label = []
    for idx, label in enumerate(labels):
        if float(label) > 0.2:  # Convert label to float before comparison
            word_label.append(labels_dict[idx])
    word_labels.append(word_label)

# Add the word labels as a new column in the DataFrame
df['Word_Labels'] = word_labels

# Save the updated DataFrame to a new CSV file
df.to_csv('predicted_file_random.csv', index=False)


# Classifying the enter dataset

In [4]:
df = pd.read_csv('/content/essayData.csv')
df.head()

# Dictionary mapping indices to labels
labels_dict = {
    0: 'technology',
    1: 'business',
    2: 'humanities',
    3: 'sciences'
}


In [5]:

# Create an empty list to store the predictions
predictions = []
# Create an empty list to store the word labels
word_labels = []

# Iterate through each row of the DataFrame
for index, row in df.iterrows():
    # Get the value from the "Section" column
    sequence_to_classify = row['ABSTRACT']
    # Define the candidate labels
    candidate_labels = ['technology', 'business', 'humanities', 'sciences']

    # Perform zero-shot classification and get the predicted label
    prediction = classifier(sequence_to_classify, candidate_labels)

    # Add the predicted label to the list
    predictions.append(prediction['scores'])
    # Adding max score to word_label coloumn
    word_label = (labels_dict[prediction['scores'].index(max(prediction['scores']))])
    word_labels.append(word_label)
    print(f'Finished classifying row :{index}/236')

# Add the predictions as a new column in the DataFrame

# Save the updated DataFrame to a new CSV file


Finished classifying row :0/236
Finished classifying row :1/236
Finished classifying row :2/236
Finished classifying row :3/236
Finished classifying row :4/236
Finished classifying row :5/236
Finished classifying row :6/236
Finished classifying row :7/236
Finished classifying row :8/236
Finished classifying row :9/236
Finished classifying row :10/236
Finished classifying row :11/236
Finished classifying row :12/236
Finished classifying row :13/236
Finished classifying row :14/236
Finished classifying row :15/236
Finished classifying row :16/236
Finished classifying row :17/236
Finished classifying row :18/236
Finished classifying row :19/236
Finished classifying row :20/236
Finished classifying row :21/236
Finished classifying row :22/236
Finished classifying row :23/236
Finished classifying row :24/236
Finished classifying row :25/236
Finished classifying row :26/236
Finished classifying row :27/236
Finished classifying row :28/236
Finished classifying row :29/236
Finished classifying

NameError: ignored

In [6]:
df['Predicted_Label'] = predictions
df['Word_Label'] = word_labels

df.to_csv('predicted_file.csv', index=False)
